In [1]:
import functions as func
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# Preprocessing

In [2]:
# Um diretorio que contém todos os datasets
path = 'data'

df_dict = func.readCSV_DATA(path)

In [3]:
df_train = df_dict['train.csv']

In [4]:
question_lst = []
answer_lst = []

for column in df_train.columns[11:]:
    match = re.match('([a-z]*)_', column)
    if match.group(1) == 'question':
        question_lst.append(column)
    else:
        answer_lst.append(column)

In [5]:
df_train = func.prepareData(df_train)

# Extracting simple text features

In [6]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [7]:
%%time
question_vectorizer = TfidfVectorizer(tokenizer=word_tokenize,
                                      strip_accents='ascii',
                                      stop_words='english',
                                      min_df=3,
                                      max_df=int(df_train.shape[0]/30))

question_X = question_vectorizer.fit_transform(df_train['question_text_clean'])

CPU times: user 5.94 s, sys: 3.62 ms, total: 5.94 s
Wall time: 6.06 s


In [8]:
%%time
answer_vectorizer = TfidfVectorizer(tokenizer=word_tokenize,
                                    strip_accents='ascii',
                                    stop_words='english',
                                    min_df=3,
                                    max_df=int(df_train.shape[0]/30))
answer_X = answer_vectorizer.fit_transform(df_train['answer_text_clean'])

CPU times: user 5.51 s, sys: 16 ms, total: 5.53 s
Wall time: 5.53 s


In [9]:
question_y = df_train[question_lst]
answer_y = df_train[answer_lst]

In [10]:
X_train_question, X_test_question, y_train_question, y_test_question = train_test_split(
    question_X, question_y, test_size=0.2)

X_train_answer, X_test_answer, y_train_answer, y_test_answer = train_test_split(
    answer_X, answer_y, test_size=0.2)

# Train a Linear Regression in NN format

## Create model - Answer

In [11]:
output_dim = len(answer_lst)
input_dim = len(answer_vectorizer.vocabulary_)

answer_model = func.create_model(output_dim, input_dim, 'binary_crossentropy')

answer_model = func.train_model(answer_model, X_train_answer, y_train_answer, X_test_answer,
                                y_test_answer, batch_size=128, nb_epoch=60, verbose=0)

In [12]:
func.spearman_corr(y_train_answer, answer_model.predict(X_train_answer.toarray()))

0.5736139166916241

In [13]:
func.spearman_corr(y_test_answer, answer_model.predict(X_test_answer.toarray()))

0.233128979166548

## Create model - Question

In [14]:
output_dim = len(question_lst)
input_dim  = len(question_vectorizer.vocabulary_)

question_model = func.create_model(output_dim, input_dim, 'binary_crossentropy')

In [15]:
question_model = func.train_model(question_model, X_train_question, y_train_question, X_test_question,
                                  y_test_question, batch_size=64, nb_epoch=100, verbose=1)

Train on 4863 samples, validate on 1216 samples
Epoch 1/100
4863/4863 [==============================] - 1s 145us/sample - loss: 0.7192 - val_loss: 0.6934
Epoch 2/100
4863/4863 [==============================] - 0s 70us/sample - loss: 0.6701 - val_loss: 0.6515
Epoch 3/100
4863/4863 [==============================] - 0s 67us/sample - loss: 0.6307 - val_loss: 0.6180
Epoch 4/100
4863/4863 [==============================] - 0s 66us/sample - loss: 0.5991 - val_loss: 0.5915
Epoch 5/100
4863/4863 [==============================] - 0s 66us/sample - loss: 0.5736 - val_loss: 0.5701
Epoch 6/100
4863/4863 [==============================] - 0s 68us/sample - loss: 0.5528 - val_loss: 0.5529
Epoch 7/100
4863/4863 [==============================] - 0s 69us/sample - loss: 0.5357 - val_loss: 0.5389
Epoch 8/100
4863/4863 [==============================] - 0s 76us/sample - loss: 0.5215 - val_loss: 0.5272
Epoch 9/100
4863/4863 [==============================] - 0s 69us/sample - loss: 0.5095 - val_loss: 0.51

4863/4863 [==============================] - 0s 65us/sample - loss: 0.3676 - val_loss: 0.4452
Epoch 78/100
4863/4863 [==============================] - 0s 64us/sample - loss: 0.3670 - val_loss: 0.4453
Epoch 79/100
4863/4863 [==============================] - 0s 64us/sample - loss: 0.3664 - val_loss: 0.4455
Epoch 80/100
4863/4863 [==============================] - 0s 64us/sample - loss: 0.3658 - val_loss: 0.4456
Epoch 81/100
4863/4863 [==============================] - 0s 64us/sample - loss: 0.3653 - val_loss: 0.4458
Epoch 82/100
4863/4863 [==============================] - 0s 64us/sample - loss: 0.3647 - val_loss: 0.4460
Epoch 83/100
4863/4863 [==============================] - 0s 72us/sample - loss: 0.3642 - val_loss: 0.4461
Epoch 84/100
4863/4863 [==============================] - 0s 63us/sample - loss: 0.3636 - val_loss: 0.4463
Epoch 85/100
4863/4863 [==============================] - 0s 64us/sample - loss: 0.3631 - val_loss: 0.4465
Epoch 86/100
4863/4863 [==========================

In [18]:
func.spearman_corr(y_train_question, question_model.predict(X_train_question.toarray()))

0.6636870012671393

In [17]:
func.spearman_corr(y_test_question, question_model.predict(X_test_question.toarray()))

0.31949589982895704

# Submission

In [28]:
df_test = func.prepareData(df_dict['test.csv'])

In [29]:
question_X = question_vectorizer.transform(df_test.question_text_clean)

answer_X = answer_vectorizer.transform(df_test.answer_text_clean)

In [30]:
sub = func.make_submission_df(df_dict['sample_submission.csv'],
                              question_X.toarray(), answer_X.toarray(),
                              question_model, answer_model)